# Beginner Tutorials - 1. ML basics 
# 7. Save and Restore Models
1. 저장 방식
- 설정
 - 설치와 임포트
 - 예제 데이터셋 받기
 - 모델 정의
- 훈련하는 동안 체크포인트 저장하기
 - 체크포인트 콜백 사용하기
 - 체크포인트 콜백 매개변수
- 체크포인트 파일
- 수동으로 가중치 저장하기
- 모델 전체를 저장하기
 - HDF5 파일로 저장하기
 - saved_model을 사용하기
- 그 다음엔

URL : https://www.tensorflow.org/alpha/tutorials/keras/save_and_restore_models<br>
GitHub : https://github.com/tensorflow/docs/blob/master/site/ko/alpha/tutorials/keras/save_and_restore_models.ipynb

# 1. 저장 방식
### 저장과 복원 관련 URL

https://www.tensorflow.org/guide/saved_model<br>
https://www.tensorflow.org/guide/eager#object-based_saving

## 2. 설정

In [1]:
!pip install -q h5py pyyaml

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

!pip install -q tensorflow==2.0.0-alpha0
import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.0.0-alpha0'

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28*28) / 255.
test_images = test_images[:1000].reshape(-1, 28*28) / 255.

### 모델 정의

In [4]:
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10, activation='softmax')
    ])
    
    model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['acc'])
    
    return model

model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## 3. 훈련하는 동안 체크포인트 저장하기

tf.keras.callbacks.ModelCheckpoint은 이런 작업을 수행하는 콜백(callback)입니다. 이 콜백은 체크포인트 작업을 조정할 수 있도록 여러가지 매개변수를 제공

### 체크포인트 콜백 사용하기

In [5]:
checkpoint_path='training_1/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                save_weights_only=True,
                                                verbose=1)

model = create_model()

model.fit(train_images, train_labels,
         epochs=10,
         validation_data = (test_images, test_labels),
         callbacks = [cp_callback])

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 640/1000 [==================>...........] - ETA: 0s - loss: 1.3636 - acc: 0.6203 
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 706us/sample - loss: 1.1209 - acc: 0.6880 - val_loss: 0.7008 - val_acc: 0.7790
Epoch 2/10
 608/1000 [=================>............] - ETA: 0s - loss: 0.4554 - acc: 0.8651
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 192us/sample - loss: 0.4179 - acc: 0.8780 - val_loss: 0.5230 - val_acc: 0.8380
Epoch 3/10
 608/1000 [=================>............] - ETA: 0s - loss: 0.2592 - acc: 0.9243
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 197us/sample - loss: 0.2865 - acc: 0.9150 - val_loss: 0.4864 - val_acc: 0.8440
Epoch 4/10
 576/1000 [================>.............] - ETA: 0s - loss: 0.2359 - acc: 0.9427
Epoch 00004: saving model to training_1/cp.ckpt
1000/1

In [6]:
#!ls {checkpoint_dir}

model = create_model()
loss, acc = model.evaluate(test_images, test_labels)
print('훈련되지 않는 모델의 정확도 :{:5.2f}%'.format(100*acc))

1000/1000 [==============================] - 0s 105us/sample - loss: 2.3369 - acc: 0.1230
훈련되지 않는 모델의 정확도 :12.30%


In [7]:
model.load_weights(checkpoint_path)
loss, acc = model.evaluate(test_images, test_labels)
print('훈련되지 않는 모델의 정확도 :{:5.2f}%'.format(100*acc))

1000/1000 [==============================] - 0s 55us/sample - loss: 0.3950 - acc: 0.8770
훈련되지 않는 모델의 정확도 :87.70%


### 체크포인트 콜백 매개변수

In [8]:
checkpoint_path = 'training_2/cp-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
checkpoint_path, verbose=1, save_weights_only=True, period=5)

model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images, train_labels,
         epochs=50, callbacks=[cp_callback],
         validation_data=(test_images,test_labels), verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [9]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2\\cp-0050.ckpt'

In [10]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 106us/sample - loss: 0.5041 - acc: 0.8700
복원된 모델의 정확도: 87.00%


## 4. 체크포인트 파일
- 모델의 가중치를 포함하는 하나 이상의 샤드(shard)
- 가중치가 어느 샤드에 저장되어 있는지를 나타내는 인덱스 파일

단일 머신에서 모델을 훈련한다면 .data-00000-of-00001 확장자를 가진 샤드 하나만 만들어 집니다.

## 5. 수동으로 가중치 저장하기

In [11]:
model.save_weights('./checkpoints/my_checkpoint')

model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss, acc = model.evaluate(test_images, test_labels)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 107us/sample - loss: 0.5041 - acc: 0.8700
복원된 모델의 정확도: 87.00%


## 6. 모델 전체를 저장하기

전체 모델을 파일 하나에 저장할 수 있습니다. 여기에는 가중치, 모델 구성 심지어 옵티마이저에 지정한 설정까지 포함됩니다.

TensorFlow.js로 모델을 로드한 다음 웹 브라우저에서 모델을 훈련하고 실행할 수 있습니다(HDF5, Saved Model). 또는 모바일 장치에 맞도록 변환한 다음 TensorFlow Lite를 사용하여 실행할 수 있습니다(HDF5, Saved Model).

### HDF5 파일로 저장하기

In [12]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

model.save('my_model.h5')

Epoch 1/5
1000/1000 [==============================] - 0s 186us/sample - loss: 1.1255 - acc: 0.6910
Epoch 2/5
1000/1000 [==============================] - 0s 87us/sample - loss: 0.4000 - acc: 0.8910
Epoch 3/5
1000/1000 [==============================] - 0s 86us/sample - loss: 0.2756 - acc: 0.9250
Epoch 4/5
1000/1000 [==============================] - 0s 90us/sample - loss: 0.2066 - acc: 0.9560
Epoch 5/5
1000/1000 [==============================] - 0s 83us/sample - loss: 0.1479 - acc: 0.9700


In [13]:
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [14]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 106us/sample - loss: 0.4504 - acc: 0.8580
복원된 모델의 정확도: 85.80%


### saved_model을 만들기

In [15]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
1000/1000 [==============================] - 0s 185us/sample - loss: 1.1542 - acc: 0.6840
Epoch 2/5
1000/1000 [==============================] - 0s 86us/sample - loss: 0.4083 - acc: 0.8860
Epoch 3/5
1000/1000 [==============================] - 0s 87us/sample - loss: 0.2819 - acc: 0.9280
Epoch 4/5
1000/1000 [==============================] - 0s 87us/sample - loss: 0.2006 - acc: 0.9510
Epoch 5/5
1000/1000 [==============================] - 0s 89us/sample - loss: 0.1582 - acc: 0.9660


In [16]:
import time
saved_model_path='./saved_model/{}'.format(int(time.time()))

tf.keras.experimental.export_saved_model(model, saved_model_path)
saved_model_path

W0411 18:12:40.948996 16216 deprecation.py:323] From C:\Users\la\Anaconda3\envs\tf20\lib\site-packages\tensorflow\python\saved_model\signature_def_utils_impl.py:253: build_tensor_info (from tensorflow.python.saved_model.utils_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
W0411 18:12:40.949994 16216 tf_logging.py:161] Export includes no default signature!
W0411 18:12:41.377849 16216 tf_logging.py:161] Export includes no default signature!


'./saved_model/1554973960'

In [17]:
new_model = tf.keras.experimental.load_from_saved_model(saved_model_path)
new_model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.predict(test_images).shape

(1000, 10)

In [19]:
new_model.compile(optimizer=model.optimizer,
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])

loss, acc = new_model.evaluate(test_images, test_labels)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 105us/sample - loss: 0.4154 - accuracy: 0.8620
복원된 모델의 정확도: 86.20%
